##### Building a Chatbot

This chatbot will only use the language model to have a conversation, There are several other related concepts that we will work on in future:
1. Conversational RAG: Enable a chatbot experience over an external source of data
2. Agents: Build a Chatbot that can take actions

In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="llama-3.3-70b-versatile",groq_api_key=groq_api_key)

c:\Users\HP\anaconda3\envs\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001F3EEED3FD0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F3EEF04070>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Gunjan and I am looking for an SDE-AI role")])
#This will give me an AIMessage which is basically a response from my LLM model


AIMessage(content="Hello Gunjan! Nice to meet you. It's great that you're looking for an SDE-AI (Software Development Engineer - Artificial Intelligence) role. That's a fascinating field, and I'm sure you must have a strong background in computer science, math, and AI/ML concepts.\n\nTo better understand your requirements, could you please tell me a bit more about your background, skills, and what you're looking for in an SDE-AI role? For example:\n\n* What type of AI/ML technologies are you interested in working with (e.g., computer vision, NLP, deep learning)?\n* Do you have any specific industry or domain in mind (e.g., healthcare, finance, autonomous vehicles)?\n* What are your preferred programming languages and tools?\n* Are you looking for a role in a specific location or are you open to remote work?\n* Do you have any specific experience or qualifications that you think would be relevant to an SDE-AI role?\n\nFeel free to share as much or as little information as you'd like, an

In [11]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hi, My name is Gunjan and I am looking for an SDE-AI role"),
    AIMessage(content="Hello Gunjan! Nice to meet you. It's great that you're looking for an SDE-AI (Software Development Engineer - Artificial Intelligence) role. That's a fascinating field, and I'm sure you must have a strong background in computer science, math, and AI/ML concepts.\n\nTo better understand your requirements, could you please tell me a bit more about your background, skills, and what you're looking for in an SDE-AI role? For example:\n\n* What type of AI/ML technologies are you interested in working with (e.g., computer vision, NLP, deep learning)?\n* Do you have any specific industry or domain in mind (e.g., healthcare, finance, autonomous vehicles)?\n* What are your preferred programming languages and tools?\n* Are you looking for a role in a specific location or are you open to remote work?\n* Do you have any specific experience or qualifications that you think would be relevant to an SDE-AI role?\n\nFeel free to share as much or as little information as you'd like, and I'll do my best to help you explore SDE-AI opportunities that fit your interests and skills!"),
    HumanMessage(content="Hi, What;s my name and what am I looking for?")
]) #Here i am hardcoding the AIMessage, the message i got from the previous cell
#I am then asking the AI whether it can remember the message i gave it before

AIMessage(content="Your name is Gunjan, and you're looking for an SDE-AI (Software Development Engineer - Artificial Intelligence) role.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 311, 'total_tokens': 338, 'completion_time': 0.032647364, 'completion_tokens_details': None, 'prompt_time': 0.01991398, 'prompt_tokens_details': None, 'queue_time': 0.16166055, 'total_time': 0.052561344}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3272ea2d91', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c0419-c15d-7c13-bf38-01975b017fe4-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 311, 'output_tokens': 27, 'total_tokens': 338})

### Message History
We can use Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input

In [17]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

#Now as different different users and simultaneously interacting with the chatbot we need to make sure that one session is completely different from the other session
#We will create a function for that with the parameter as session id which is a string
#The resturn type of this function would be BaseChatMessageHistory--->whatever chat history is created it is imported from this library

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory: #this session id will be used to distinguish between sessions
    if session_id not in store:
        store[session_id]=ChatMessageHistory() #this is an object of chat message history
        #we are storing the session id in the store dictionary
        #With respect to the session id value i am initializing a chat message history
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)
#this ensures that we can interact with our LLM model based on our chat history

In [18]:
config={"configurable":{"session_id":"chat1"}} #here i am hardcoding the session id

In [20]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Gunjan and I am looking for an SDE-AI role")],
    config=config
)

In [21]:
response.content

"Hi Gunjan, nice to meet you! I'd be happy to help you with your job search for an SDE-AI (Software Development Engineer - Artificial Intelligence) role.\n\nTo get started, can you tell me a bit about your background and experience? For example:\n\n* What type of degree do you have (e.g. Bachelor's, Master's, Ph.D.) and in what field (e.g. Computer Science, Mathematics, Engineering)?\n* Do you have any prior experience working with AI or ML technologies (e.g. internships, projects, research experience)?\n* What programming languages are you proficient in (e.g. Python, Java, C++).\n* Are you familiar with any deep learning frameworks (e.g. TensorFlow, PyTorch, Keras)?\n\nAlso, what specific areas of AI are you interested in (e.g. computer vision, natural language processing, robotics)?\n\nLastly, what are your expectations from an SDE-AI role (e.g. job responsibilities, company culture, growth opportunities)? \n\nThis will help me understand your profile better and provide more tailored

In [22]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config
)

AIMessage(content='Your name is Gunjan.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 544, 'total_tokens': 551, 'completion_time': 0.006888205, 'completion_tokens_details': None, 'prompt_time': 0.027596556, 'prompt_tokens_details': None, 'queue_time': 0.052730521, 'total_time': 0.034484761}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c0429-846d-7df3-89b6-5a0bb99e5de3-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 544, 'output_tokens': 7, 'total_tokens': 551})

In [ ]:
#Change the config--->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="What's my name")],
    config=config1
)
response.content

"I don't know your name. I'm a large language model, I don't have the ability to recall personal information about individuals, and our conversation just started, so I haven't been given any information about you. If you'd like to share your name, I'd be happy to chat with you!"

In [26]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi, my name if Gina")],
    config=config1
)
response.content

"Nice to meet you, Gina! It's lovely to have you here. Is there something I can help you with or would you like to chat? I'm all ears!"

In [27]:

response=with_message_history.invoke(
    [HumanMessage(content="What's my name")],
    config=config1
)
response.content

'I remember! Your name is Gina. We just introduced ourselves a moment ago.'